### Hunting detection #4


In [1]:
import sys,os
os.chdir('../../')
sys.path += [ os.path.abspath('../str-omega'), os.path.abspath('../str-omega/sigma/tools')]


## Import Libraries and convert detection to SIEM query

In [2]:
import json
from sigma.config.collection import SigmaConfigurationManager
from sigma.configuration import SigmaConfiguration, SigmaConfigurationChain
from sigma.parser.collection import SigmaCollectionParser
from sigma.backends.elasticsearch import ElastalertBackendQs

from sigma.backends.elasticsearch import ElasticsearchDSLBackend


class ConvertDetection(ElasticsearchDSLBackend):
    def __init__(self, *args, **kwargs):
        mapping, = args
        with open(mapping, 'r') as stream:
            self.sigmaconfigs = SigmaConfiguration(configyaml=stream)
        super().__init__(self.sigmaconfigs, {})

    def convert(self, a2b_file):
        with open(a2b_file, 'r') as stream:
            parser = SigmaCollectionParser(stream, self.sigmaconfigs, None)

        parser.generate(self)
        result = self.finalize()
        return result
        
mapping = "./Demo/RT-TOP5/detections/mapping.yml"
detection =  "./Demo/RT-TOP5/detections/detection4.yml"

backend = ConvertDetection(mapping)
query_txt = backend.convert(detection)
query = json.loads(query_txt)
print(json.dumps(
    query,
    sort_keys=True,
    indent=4,
    separators=(',', ': ')
))


{
    "query": {
        "constant_score": {
            "filter": {
                "bool": {
                    "must": [
                        {
                            "match_phrase": {
                                "event_id": "1"
                            }
                        },
                        {
                            "bool": {
                                "should": [
                                    {
                                        "wildcard": {
                                            "process_command_line.keyword": "*/f*"
                                        }
                                    },
                                    {
                                        "wildcard": {
                                            "process_command_line.keyword": "*-f*"
                                        }
                                    }
                                ]
                            }
               

## Create connection to SIEM and run converted query

In [3]:
#!python3 -m pip install elasticsearch
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://localhost:9200'])
res = es.search(index="logs-endpoint-winevent-sysmon-*", body=query, size='10000')
print("Got %d Hits" % res['hits']['total']['value'])

Got 2945 Hits


In [ ]:
# res['hits']

## Check results 

In [4]:
from pandas import json_normalize
import pandas as pd
#pd.set_option('display.max_colwidth', None)

df = json_normalize([x['_source'] for x in res['hits']['hits']])
df[['event_original_time', 'process_command_line']]
# with pd.option_context('display.max_rows', 10, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
#     df_styled = df.copy()
#     df_styled = df_styled[['event_original_time', 'process_command_line']]
#     df_styled = df_styled.stack().str.lstrip().unstack()
#     df_styled = df_styled.style.set_properties(**{'text-align': 'left'})
    
# df_styled

,event_original_time,process_command_line
0,2021-07-27T00:38:43.359Z,\??\c:\windows\system32\conhost.exe 0xffffffff...
1,2021-07-27T00:38:44.548Z,"""collectguestlogs.exe"" -mode:ga -filename:d:\c..."
2,2021-07-27T00:38:44.565Z,\??\c:\windows\system32\conhost.exe 0xffffffff...
3,2021-07-27T00:38:53.531Z,\??\c:\windows\system32\conhost.exe 0xffffffff...
4,2021-07-27T00:29:37.057Z,\??\c:\windows\system32\conhost.exe 0xffffffff...
...,...,...
2940,2021-08-03T13:27:34.379Z,"c:\windows\system32\taskkill.exe /f /fi ""modul..."
2941,2021-08-03T13:27:34.387Z,\??\c:\windows\system32\conhost.exe 0xffffffff...
2942,2021-08-03T13:27:34.564Z,\??\c:\windows\system32\conhost.exe 0xffffffff...
2943,2021-08-03T13:27:33.881Z,\??\c:\windows\system32\conhost.exe 0xffffffff...


In [6]:
def calc(cmdline):
    return len(cmdline) - len(cmdline.encode("ascii", "ignore"))

df['obf'] = df.apply(lambda row : calc(row['process_command_line']), axis = 1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    df_styled = df[['event_original_time', 'process_command_line', 'obf']].sort_values(by=['obf'], ascending=False).head(5).copy()
    df_styled = df_styled[['event_original_time', 'process_command_line']]
    df_styled = df_styled.stack().str.lstrip().unstack()
    df_styled = df_styled.style.set_properties(**{'text-align': 'left'})
    
df_styled




,event_original_time,process_command_line
2851,2021-08-03T08:51:04.538Z,"certutil -f -₞u₞rᴸ""c₞a₞ch""e -split https://wietze.github.io/robots.txt output.txt"
0,2021-07-27T00:38:43.359Z,\??\c:\windows\system32\conhost.exe 0xffffffff -forcev1
1967,2021-07-30T08:45:29.361Z,c:\windows\system32\compattelrunner.exe -m:appraiser.dll -f:updateavstatus -cv:c9kfrkhweu2+g8u3.3
1958,2021-07-30T05:00:14.730Z,\??\c:\windows\system32\conhost.exe 0xffffffff -forcev1
1959,2021-07-30T05:00:22.594Z,\??\c:\windows\system32\conhost.exe 0xffffffff -forcev1
